### Importing Needed Libraries
- Pandas (pd)
- NumPy (np)

In [2]:
import pandas as pd
import numpy as np

### Importing Data
- Path: "./hotel_bookings_data.csv"

In [3]:
df = pd.read_csv("./hotel_bookings_data.csv")
df.head()
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

### 1. Cleaning
- Missing Data
- Duplicates

In [4]:
unnecessary_columns = ["meal", "distribution_channel", "adults", "children", "babies", "required_car_parking_spaces",
                        "reservation_status", "reservation_status_date", "total_of_special_requests"]
df.drop(columns=unnecessary_columns, inplace=True)

In [5]:
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'country', 'market_segment',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr'],
      dtype='object')

In [6]:
df = df[~df["country"].isna()]

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 85404 entries, 0 to 119389
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           85404 non-null  object 
 1   is_canceled                     85404 non-null  int64  
 2   lead_time                       85404 non-null  int64  
 3   arrival_date_year               85404 non-null  int64  
 4   arrival_date_month              85404 non-null  object 
 5   arrival_date_week_number        85404 non-null  int64  
 6   arrival_date_day_of_month       85404 non-null  int64  
 7   stays_in_weekend_nights         85404 non-null  int64  
 8   stays_in_week_nights            85404 non-null  int64  
 9   country                         85404 non-null  object 
 10  market_segment                  85404 non-null  object 
 11  is_repeated_guest               85404 non-null  int64  
 12  previous_cancellations          8540

### 2. Memory Optimization

In [9]:
(df.memory_usage(deep=True).sum() / (1024**2)).round(2)

np.float64(46.46)

In [10]:
new_dt = {
    'hotel':                            "category",
    'market_segment':                   "category",
    'arrival_date_month':               "category",
    'reserved_room_type':               "category",
    'deposit_type':                     "category",
    'customer_type':                    "category",
    'country':                          "category",
    'assigned_room_type':               "category",
    'agent':                            "category",
    'company':                          "category",
    'is_canceled':                      "bool",
    'is_repeated_guest':                "bool",
    'adr':                              "float64",
    'lead_time':                        "int16",
    'arrival_date_year':                "int16",
    'days_in_waiting_list':             "int16",
    'arrival_date_week_number':         "int8",
    'arrival_date_day_of_month':        "int8",
    'stays_in_weekend_nights':          "int8",
    'stays_in_week_nights':             "int8",
    'previous_cancellations':           "int8",
    'previous_bookings_not_canceled':   "int8",
    'booking_changes':                  "int8"
}

df = df.astype(new_dt)

In [11]:
(df.memory_usage(deep=True).sum() / (1024**2)).round(2)

np.float64(3.62)

### EDA
- Total Bookings
- Total Cancellitions
- Total Revenue

In [12]:
len(df)

85404

In [13]:
df["is_canceled"].sum()

np.int64(23450)

In [14]:
(df["adr"] * (df["stays_in_week_nights"] + df["stays_in_weekend_nights"])).sum()

np.float64(33848078.69)

### 3. Revenue Analytics
- ADR per hotel type
- ADR per month
- ADR per assigned room type
- ADR per reserved room type
- Ranking months by total revenue

In [15]:
df["revenue"] = (df["adr"] * (df["stays_in_week_nights"] + df["stays_in_weekend_nights"]))

In [16]:
df.groupby("hotel", as_index=False, observed=False)["revenue"].sum()

,hotel,revenue
0,City Hotel,18458519.43
1,Resort Hotel,15389559.26


In [17]:
df.groupby("assigned_room_type", as_index=False, observed=False)["adr"].sum()

,assigned_room_type,adr
0,A,4339428.74
1,B,169784.69
2,C,249527.38
3,D,2403286.04
4,E,842878.03
5,F,548184.97
6,G,414580.68
7,H,120891.36
8,I,14477.87
9,K,14856.99


In [18]:
df.groupby("reserved_room_type", as_index=False, observed=False)["adr"].sum()

,reserved_room_type,adr
0,A,5103084.29
1,B,87531.22
2,C,145863.39
3,D,2089112.20
4,E,749650.73
5,F,470294.76
6,G,359239.82
7,H,112380.34
8,L,748.00
9,P,0.00


In [19]:
df.groupby("assigned_room_type", as_index=False, observed=False)["adr"].sum()

,assigned_room_type,adr
0,A,4339428.74
1,B,169784.69
2,C,249527.38
3,D,2403286.04
4,E,842878.03
5,F,548184.97
6,G,414580.68
7,H,120891.36
8,I,14477.87
9,K,14856.99


In [20]:
df.groupby("arrival_date_month", as_index=False, observed=False)["revenue"].sum().sort_values("revenue", ascending=False)

,arrival_date_month,revenue
1,August,7123705.99
5,July,5771961.42
6,June,3444849.57
8,May,3057274.12
0,April,2731522.98
11,September,2618899.21
7,March,2015478.71
10,October,1992010.80
3,February,1455720.14
2,December,1376903.62


### 4. Monthly Occupincy Trends
- peak booking months for City Hotels based on Occupancy Rate (%)
- peak booking months for Resort Hotels based on Occupancy Rate (%)
- peak booking months for City Hotels based on Total Bookings
- peak booking months for Resort Hotels based on Total Bookings
- peak booking months for City Hotels based on Total Booked Room-Nights
- peak booking months for Resort Hotels based on Total Booked Room-Nights

In [21]:
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

hotel_types_details = df.loc[:, ["arrival_date_month", "hotel", "adr", "stays_in_week_nights", "stays_in_weekend_nights"]]
hotel_types_details["total_revenue"] = hotel_types_details["adr"] * (hotel_types_details["stays_in_week_nights"] + hotel_types_details["stays_in_weekend_nights"])
hotel_types_details["bookings"] = 1
hotel_types_details = (hotel_types_details
                        .groupby(
                            ["hotel", "arrival_date_month"],
                            as_index=False,
                            observed=False
                        ).agg({
                            "adr": "sum",
                            "stays_in_week_nights": "sum",
                            "stays_in_weekend_nights": "sum",
                            "total_revenue": "sum",
                            "bookings": "sum"
                        })
                    )
conds = []
for x in months:
    conds.append((hotel_types_details["arrival_date_month"] == x))
vals = []
for x in months:
    if x in ['January', 'March', 'May', 'July', 'August', 'October', 'December']:
        vals.append(31)
    elif x == 'February':
        vals.append(28)
    else:
        vals.append(30)
hotel_types_details["days_of_month"] = np.select(conds, vals)
hotel_types_details["total_available_capacity"] = np.where(hotel_types_details["hotel"] == "City Hotel", 1000, 600) * hotel_types_details["days_of_month"]
hotel_types_details.sample()

,hotel,arrival_date_month,adr,stays_in_week_nights,stays_in_weekend_nights,total_revenue,bookings,days_of_month,total_available_capacity
17,Resort Hotel,July,663510.44,16242,6372,3511681.63,4234,31,18600


In [22]:
occupincy = (
    (
        (
            (hotel_types_details["stays_in_week_nights"]
                + hotel_types_details["stays_in_weekend_nights"]
            ) / hotel_types_details["total_available_capacity"]
        ) * 100
    ).round(2)
)
(hotel_types_details
    .loc[
        hotel_types_details["hotel"] == "City Hotel",
        ["arrival_date_month"]
    ].assign(
        occupincy_rate= occupincy
    ).sort_values("occupincy_rate",
                    ascending=False,
                    ignore_index=True
    )
)

,arrival_date_month,occupincy_rate
0,August,70.93
1,July,61.13
2,April,52.16
3,May,50.12
4,June,48.99
5,March,48.93
6,February,40.45
7,September,40.10
8,October,38.94
9,December,31.19


In [23]:
occupincy = (
    (
        (
            (hotel_types_details["stays_in_week_nights"]
                + hotel_types_details["stays_in_weekend_nights"]
            ) / hotel_types_details["total_available_capacity"]
        ) * 100
    ).round(2)
)
(hotel_types_details
    .loc[
        hotel_types_details["hotel"] == "Resort Hotel",
        ["arrival_date_month"]
    ].assign(
        occupincy_rate= occupincy
    ).sort_values("occupincy_rate",
                    ascending=False,
                    ignore_index=True
    )
)

,arrival_date_month,occupincy_rate
0,August,129.44
1,July,121.58
2,June,81.78
3,September,69.83
4,May,69.27
5,April,61.01
6,October,55.81
7,March,53.26
8,February,47.10
9,November,40.02


In [24]:
(hotel_types_details
    .loc[
        hotel_types_details["hotel"] == "City Hotel",
        ["arrival_date_month", "bookings"]
    ].sort_values("bookings",
                    ascending=False,
                    ignore_index=True
    )
)

,arrival_date_month,bookings
0,August,6487
1,July,5639
2,May,5278
3,April,4967
4,June,4927
5,March,4787
6,September,4144
7,October,4121
8,February,3567
9,December,2940


In [25]:
(hotel_types_details
    .loc[
        hotel_types_details["hotel"] == "Resort Hotel",
        ["arrival_date_month", "bookings"]
    ].sort_values("bookings",
                    ascending=False,
                    ignore_index=True
    )
)

,arrival_date_month,bookings
0,August,4587
1,July,4234
2,May,2872
3,April,2744
4,June,2705
5,October,2612
6,March,2566
7,February,2388
8,September,2376
9,December,2032


In [26]:
(hotel_types_details
    .loc[
        hotel_types_details["hotel"] == "City Hotel",
        ["arrival_date_month"]
    ].assign(
        total_booked_room_nights= hotel_types_details["stays_in_week_nights"] + hotel_types_details["stays_in_weekend_nights"]
    ).sort_values("total_booked_room_nights",
                    ascending=False,
                    ignore_index=True
    )
)

,arrival_date_month,total_booked_room_nights
0,August,21987
1,July,18951
2,April,15648
3,May,15538
4,March,15168
5,June,14697
6,October,12072
7,September,12031
8,February,11327
9,December,9670


In [27]:
(hotel_types_details
    .loc[
        hotel_types_details["hotel"] == "Resort Hotel",
        ["arrival_date_month"]
    ].assign(
        total_booked_room_nights= hotel_types_details["stays_in_week_nights"] + hotel_types_details["stays_in_weekend_nights"]
    ).sort_values("total_booked_room_nights",
                    ascending=False,
                    ignore_index=True
    )
)

,arrival_date_month,total_booked_room_nights
0,August,24075
1,July,22614
2,June,14720
3,May,12885
4,September,12570
5,April,10981
6,October,10381
7,March,9906
8,February,7913
9,November,7203


### 5. Lead Time Analytics
- Calculate the Average Lead Time by customer_type
- Calculate the Average Lead Time by month
- Calculate the Average Lead Time by hotel type
- Calculate the Average Lead Time by country
- Calculate the Average Lead Time by market segment
- Calculate the Average Lead Time by assigned room type
- Calculate the Average Lead Time by reserved room type

In [37]:
def alt_by_field(field):
    return (df
    .groupby(field, as_index=False, observed=False)
    .agg({"lead_time": "mean"})
    .sort_values("lead_time", ascending=False, ignore_index=True)
    .round(2)
)

In [38]:
alt_by_field("customer_type")

,customer_type,lead_time
0,Transient-Party,111.57
1,Contract,108.97
2,Transient,73.49
3,Group,52.09


In [39]:
alt_by_field("arrival_date_month")

,arrival_date_month,lead_time
0,July,111.55
1,June,103.13
2,August,102.50
3,September,93.38
4,May,91.27
5,October,82.08
6,April,75.71
7,December,59.17
8,March,53.37
9,November,47.46


In [40]:
alt_by_field("hotel")

,hotel,lead_time
0,Resort Hotel,83.67
1,City Hotel,77.05


In [41]:
alt_by_field("country")

,country,lead_time
0,FJI,322.00
1,BEN,274.00
2,LCA,268.00
3,FRO,267.33
4,KNA,251.50
...,...,...
172,AIA,0.00
173,DJI,0.00
174,MDG,0.00
175,MLI,0.00


In [42]:
alt_by_field("market_segment")

,market_segment,lead_time
0,Groups,146.74
1,Offline TA/TO,105.48
2,Online TA,79.60
3,Direct,49.07
4,Corporate,16.61
5,Complementary,13.76
6,Aviation,4.28
7,Undefined,1.50


In [43]:
alt_by_field("assigned_room_type")

,assigned_room_type,lead_time
0,B,91.97
1,E,83.08
2,A,82.70
3,C,82.51
4,G,75.63
5,H,75.14
6,D,74.08
7,F,67.42
8,I,67.04
9,K,42.58


In [45]:
alt_by_field("reserved_room_type")

,reserved_room_type,lead_time
0,B,114.57
1,E,87.92
2,D,82.67
3,G,79.71
4,C,78.63
5,H,78.12
6,A,77.76
7,F,68.18
8,L,0.00
9,P,0.00


### 6. Cancellition Analytics
- Overall Cancellation Rate
- Cancellation Rate by Customer Type
- Cancellation Rate by Hotel Type
- Cancellation Rate by Country
- Cancellation Rate by Market Segment
- Cancellation Rate by Lead Time
- Cancellation Rate by Reserved Room Type and Assigned Room Type

In [47]:
(df["is_canceled"].sum() / len(df)).round(2)

np.float64(0.27)

In [66]:
def cancellition_rate_by_fields(fields):
    temp = (df
            .assign(count= 1)
            .groupby(fields, observed=False)
            .agg({"is_canceled": "sum", "count": "sum"})
            .reset_index()
    )
    return (temp
            .assign(cancellition_rate= ((temp["is_canceled"] / temp["count"]) * 100).round(2))
            .sort_values("cancellition_rate", ascending=False, ignore_index=True)
            .drop(columns= ["is_canceled", "count"])
    )

In [67]:
cancellition_rate_by_fields("customer_type")

,customer_type,cancellition_rate
0,Transient,30.09
1,Contract,16.31
2,Transient-Party,14.67
3,Group,9.96


In [68]:
cancellition_rate_by_fields("hotel")

,hotel,cancellition_rate
0,City Hotel,29.87
1,Resort Hotel,23.62


In [69]:
cancellition_rate_by_fields("country")

,country,cancellition_rate
0,BEN,100.0
1,MYT,100.0
2,KHM,100.0
3,JEY,100.0
4,IMN,100.0
...,...,...
172,SLE,0.0
173,SMR,0.0
174,SLV,0.0
175,SUR,0.0


In [70]:
cancellition_rate_by_fields("market_segment")

,market_segment,cancellition_rate
0,Undefined,100.00
1,Online TA,35.24
2,Groups,26.81
3,Aviation,19.82
4,Direct,14.79
5,Offline TA/TO,14.70
6,Complementary,12.50
7,Corporate,12.11


In [71]:
cancellition_rate_by_fields("lead_time")

,lead_time,cancellition_rate
0,521,100.0
1,629,100.0
2,626,100.0
3,622,100.0
4,615,100.0
...,...,...
474,460,0.0
475,463,0.0
476,445,0.0
477,709,0.0


In [76]:
cancellition_rate_by_fields(["assigned_room_type", "reserved_room_type"])

,assigned_room_type,reserved_room_type,cancellition_rate
0,P,P,100.00
1,L,L,100.00
2,C,L,100.00
3,H,H,41.87
4,G,G,36.75
...,...,...,...
115,P,E,NaN
116,P,F,NaN
117,P,G,NaN
118,P,H,NaN


### 7. Customer Analysis
- Bookings by customer type
- Bookings by agent

In [85]:
def bookings_by_field(field):
        temp = (df
                .assign(count= 1)
                .groupby(field, as_index=False, observed=False)
                .agg({"is_canceled": "sum", "count": "sum"})
                
        )
        return (temp
                .assign(total_bookings= temp["count"] - temp["is_canceled"])
                .drop(columns=["is_canceled", "count"])
                .sort_values("total_bookings", ascending=False, ignore_index=True)
        )

In [86]:
bookings_by_field("customer_type")

,customer_type,total_bookings
0,Transient,49395
1,Transient-Party,9474
2,Contract,2606
3,Group,479


In [87]:
bookings_by_field("agent")

,agent,total_bookings
0,9.0,16993
1,240.0,7939
2,7.0,2832
3,14.0,2739
4,250.0,2259
...,...,...
327,93.0,0
328,60.0,0
329,55.0,0
330,41.0,0
